# Simple Methods to Update v6 (and earlier?) Arches Packakage Graph Data to v7

Arches version 7 introduced internationalization features which require some update of Arches v6 graph JSON files. The functions here can be used to copy files from an Arches v6 package directory to make an Arches v7 version of the package. This includes functions to update JSON files in the `/graph/`folder so certain string values get converted into language keyed objects as expected for Arches V7.

This only updates card and graph information. No other changes are made.

This notebook demonstrates the functions with graphs from the ARM repository (see: https://github.com/archesproject/arm)

Navigate over to `files/arches-pkg-demo` to review the original and updated package files. 


In [1]:
import json
import os
import shutil

# Get the root_path for this jupyter notebook repo.
repo_path = os.path.dirname(os.path.abspath(os.getcwd()))

# Source of v6 Arches package files
SOURCE_DIR = os.path.join(
    repo_path, 'files', 'arches-pkg-demo', 'v6-pkg'
)

# Destination for v7 updated package files
UPDATE_DIR = os.path.join(
    repo_path, 'files', 'arches-pkg-demo', 'v7-pkg'
)


# Keys for 'graph' objects where v6 strings that need v7 language objects
GRAPH_OBJ_KEYS_NEEDING_LANG_OBJS = [
    'name',
    'subtitle',
    'description',
]

# Keys for 'card' objects where v6 strings that need v7 language objects
CARD_OBJ_KEYS_NEEDING_LANG_OBJS = [
    'description',
    'helptext',
    'helptitle',
    'instructions',
    'name',
    'description',
]


def copy_source_data(source_dir=SOURCE_DIR, update_dir=UPDATE_DIR):
    if not os.path.exists(update_dir):
        os.makedirs(update_dir)
    for dirpath, _, filenames in os.walk(source_dir):
        for filename in filenames:
            if filename.endswith(':Zone.Identifier'):
                # A convenience hack for Windows subsystem for linux
                continue
            if filename.startswith('.'):
                # A convenience hack for skipping git files
                continue
            update_dir_path = dirpath.replace(source_dir, update_dir)
            if not os.path.exists(update_dir_path):
                os.makedirs(update_dir_path)
            orig_file_path = os.path.join(dirpath, filename)
            new_file_path = os.path.join(update_dir_path, filename)
            print(f'Copy {orig_file_path} -> {new_file_path}')
            shutil.copy2(orig_file_path, new_file_path)



def write_json_file(obj, path):
    with open(path, "w") as outfile:
        json.dump(obj, outfile, indent=4)


def read_json_file(path):
    with open(path) as outfile:
        obj = json.load(outfile)
    return obj


def update_card_obj(card_obj):
    if not isinstance(card_obj, dict):
        return card_obj
    for key in CARD_OBJ_KEYS_NEEDING_LANG_OBJS:
        key_val = card_obj.get(key)
        if isinstance(key_val, dict):
            # this is already a dict
            continue
        elif isinstance(key_val, str):
            # Make the string value in a language obj
            key_val = {
                'en': key_val,
            }
        if not key_val:
            # this key is not present in this card obj,
            # so don't do anything with it yet
            key_val = {
                'en': None,
            }
        card_obj[key] = key_val
    return card_obj


def update_card_items(obj):
    graph_list = obj.get('graph', [])
    if not graph_list:
        return obj
    for graph_obj in graph_list:
        for card_obj in graph_obj.get('cards', []):
            # Mutate that dict!
            card_obj = update_card_obj(card_obj)
    return obj


def update_graph_obj(graph_obj):
    for key in GRAPH_OBJ_KEYS_NEEDING_LANG_OBJS:
        key_val = graph_obj.get(key)
        if isinstance(key_val, dict):
            # this is already a dict
            continue
        elif isinstance(key_val, str):
            # Make the string value in a language obj
            key_val = {
                'en': key_val,
            }
        if not key_val:
            # this key is not present in this card obj,
            # so don't do anything with it yet
            key_val = {
                'en': None,
            }
        graph_obj[key] = key_val
    return graph_obj


def update_graph_objs(obj):
    graph_list = obj.get('graph', [])
    if not graph_list:
        return obj
    for graph_obj in graph_list:
        graph_obj = update_graph_obj(graph_obj)
    return obj


def update_graph_json_file(filepath):
    if not filepath.endswith('.json'):
        return None
    obj = read_json_file(filepath)
    obj = update_graph_objs(obj)
    obj = update_card_items(obj)
    write_json_file(obj,filepath)


def update_graphs_json_files(update_dir=UPDATE_DIR):
    for dirpath, _, filenames in os.walk(update_dir):
        if not 'graphs' in dirpath:
            continue
        for filename in filenames:
            if filename.endswith(':Zone.Identifier'):
                # A convenience hack for Windows subsystem for linux
                continue
            if filename.startswith('.'):
                # A convenience hack for skipping git files
                continue
            if not filename.endswith('.json'):
                continue
            filepath = os.path.join(dirpath, filename)
            update_graph_json_file(filepath)
            print(f'Updated: {filepath}')



def make_upgraded_package_copy(source_dir=SOURCE_DIR, update_dir=UPDATE_DIR):
    # First copy the files over to the updated directory, keeps the same
    # directory stucture, and the files aren't altered
    copy_source_data(source_dir=source_dir, update_dir=update_dir)
    # Now update the JSON files in the package/graph folder
    update_graphs_json_files(update_dir=update_dir)


In [2]:
make_upgraded_package_copy()

Copy /home/ekansa/github/open-context-jupyter/files/arches-pkg-demo/v6-pkg/package_config.json -> /home/ekansa/github/open-context-jupyter/files/arches-pkg-demo/v7-pkg/package_config.json
Copy /home/ekansa/github/open-context-jupyter/files/arches-pkg-demo/v6-pkg/README.txt -> /home/ekansa/github/open-context-jupyter/files/arches-pkg-demo/v7-pkg/README.txt
Copy /home/ekansa/github/open-context-jupyter/files/arches-pkg-demo/v6-pkg/extensions/widgets/sample-widget/sample-widget.json -> /home/ekansa/github/open-context-jupyter/files/arches-pkg-demo/v7-pkg/extensions/widgets/sample-widget/sample-widget.json
Copy /home/ekansa/github/open-context-jupyter/files/arches-pkg-demo/v6-pkg/extensions/widgets/sample-widget/sample-widget.js -> /home/ekansa/github/open-context-jupyter/files/arches-pkg-demo/v7-pkg/extensions/widgets/sample-widget/sample-widget.js
Copy /home/ekansa/github/open-context-jupyter/files/arches-pkg-demo/v6-pkg/extensions/widgets/sample-widget/sample-widget.htm -> /home/ekansa/